In [1]:
import sys
import pickle
import pandas as pd
import numpy as np

In [2]:
filename = "co_T0_T1_toshare.pkl"
area_to_minority = "OAtoMinorityData.CSV"

In [3]:
pickle_in = open(filename,"rb")
df = pickle.load(pickle_in)

In [4]:
from IPython.display import display
pd.options.display.max_columns = None
df.head()

,CompanyName,CompanyNumber,RegAddressCareOf,RegAddressPOBox,RegAddressAddressLine1,RegAddressAddressLine2,RegAddressPostTown,RegAddressCounty,RegAddressCountry,RegAddressPostCode,CompanyCategory,CompanyStatus_T0,CountryOfOrigin,DissolutionDate,IncorporationDate,AccountsAccountRefDay,AccountsAccountRefMonth,AccountsNextDueDate,AccountsLastMadeUpDate,AccountsAccountCategory,ReturnsNextDueDate,ReturnsLastMadeUpDate,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,MortgagesNumMortSatisfied,LimitedPartnershipsNumGenPartners,LimitedPartnershipsNumLimPartners,PreviousName_1CONDATE,PreviousName_1CompanyName,PreviousName_2CONDATE,PreviousName_2CompanyName,PreviousName_3CONDATE,PreviousName_3CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,nSIC,SIC1,SIC2,SIC3,eCompanyCategory,eCompanyStatus,eCountryOfOrigin,eAccountsAccountCategory,dIncorporationDate,dAccountsNextDueDate,dAccountsLastMadeUpDate,dReturnsNextDueDate,dReturnsLastMadeUpDate,dConfStmtNextDueDate,dConfStmtLastMadeUpDate,dPreviousName_1CONDATE,dPreviousName_2CONDATE,namechanged,namechanged2,pcd,oseast1m,osnrth1m,oa11,cty,ru11ind,oac11,lat,long,imd,country,oac1,oac2,imdu,CompanyStatus_T1,class_1
0,! LTD,08209948,NaN,NaN,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,NaN,LS10 2RU,Private Limited Company,Active,United Kingdom,NaN,11/09/2012,30.0,9.0,30/06/2018,30/09/2016,DORMANT,09/10/2016,11/09/2015,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2019,11/09/2017,99999.0,99.0,9.0,99.0,15,2,182,3,2012.857923,2018.495890,2016.748634,2016.693989,2015.857534,2019.734247,2017.857534,NaN,NaN,0,0,LS102RU,431572.0,430988.0,E00057795,E99999999,A1,8A1,53.774278,-1.522430,5831.0,E,8,8A,5648.931384,Active,False
1,!NNOV8 LIMITED,11006939,NaN,NaN,C/O FRANK HIRTH 1ST FLOOR,236 GRAY'S INN ROAD,LONDON,NaN,UNITED KINGDOM,WC1X 8HB,Private Limited Company,Active,United Kingdom,NaN,11/10/2017,31.0,3.0,11/07/2019,NaN,NO ACCOUNTS FILED,08/11/2018,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,24/10/2019,NaN,62090.0,62.0,0.0,90.0,15,2,182,10,2017.860274,2019.852055,NaN,2018.610959,NaN,2019.813699,NaN,NaN,NaN,0,0,WC1X8HB,530883.0,182219.0,E00004531,E99999999,A1,3B2,51.523730,-0.114790,14970.0,E,3,3B,14502.572940,Active,False
2,!NSPIRED LTD,SC421617,NaN,NaN,26 POLMUIR ROAD,NaN,ABERDEEN,NaN,UNITED KINGDOM,AB11 7SY,Private Limited Company,Active,United Kingdom,NaN,11/04/2012,30.0,3.0,30/12/2018,30/03/2017,TOTAL EXEMPTION FULL,09/05/2017,11/04/2016,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,25/04/2020,11/04/2017,70229.0,70.0,2.0,29.0,15,2,182,13,2012.844262,2018.997260,2017.243836,2017.679452,2016.844262,2020.316940,2017.843836,NaN,NaN,0,0,AB117SY,393776.0,804937.0,S00090544,S99999999,1,2C3,57.135299,-2.104462,6772.0,S,2,2C,6772.000000,Active,False
3,!NVERTD DESIGNS LIMITED,09152972,NaN,NaN,55A HIGH STREET,NaN,SILSOE,BEDFORDSHIRE,NaN,MK45 4EW,Private Limited Company,Active,United Kingdom,NaN,30/07/2014,31.0,7.0,30/04/2019,31/07/2017,N/A,27/08/2016,30/07/2015,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,13/08/2020,30/07/2017,58190.0,58.0,1.0,90.0,15,2,182,9,2014.578082,2019.328767,2017.580822,2016.655738,2015.578082,2020.617486,2017.578082,NaN,NaN,0,0,MK454EW,508137.0,235898.0,E00088094,E99999999,F1,1C3,52.010987,-0.425843,29038.0,E,1,1C,28131.310156,Active,False
4,!OBAC UK LIMITED,07687209,NaN,NaN,ENDEAVOUR HOUSE,COOPERS END ROAD,STANSTED AIRPORT,NaN,NaN,CM24 1SJ,Private Limited Company,Active,United Kingdom,NaN,29/06/2011,29.0,12.0,29/09/2018,31/12/2016,SMALL,27/07/2017,29/06/2016,1,1,0,0,0,0,15/07/2011,!OBAC FITTINGS LIMITED,NaN,NaN,NaN,NaN,13/07/2020,29/06/2017,70229.0,70.0,2.0,29.0,15,2,182,12,2011.493151,2018.745205,2017.000000,2017.569863,2016.494536,2020.532787,2017.493151,2011.536986,NaN,1,0,CM241SJ,554894.0,222515.0,E00112246,E10000012,E1,5B3,51.879690,0.248905,17104.0,E,5,5B,16569.940385,Active,False


In [5]:
#keep only those companies whose T0 status is active
df = df[df['CompanyStatus_T0']=='Active']

In [6]:
features = df[['CompanyNumber', 'oac2',
        'CountryOfOrigin','MortgagesNumMortCharges',
       'MortgagesNumMortOutstanding', 'MortgagesNumMortPartSatisfied', 'LimitedPartnershipsNumGenPartners',
       'LimitedPartnershipsNumLimPartners',  'SIC1',
       'eCompanyCategory', 'eCompanyStatus',
       'eAccountsAccountCategory', 'dIncorporationDate',
       'dAccountsNextDueDate',
       'dReturnsNextDueDate','dConfStmtNextDueDate','namechanged', 'namechanged2', 'class_1']].copy()
#Remove the rows with missing data 
features.dropna(how='any',inplace=True)

In [7]:
#Create new features(periods)
T0 = 2018+(31+28)/365
T1 = 2019+(31+28)/365
features['dAccountsOutstandingTime'] = T0-features['dAccountsNextDueDate']
features['dConfStmtOutstandingTime'] = T0-features['dConfStmtNextDueDate']
features['dReturnsOutstandingTime'] = T0-features['dReturnsNextDueDate']
features['OperatingTime'] = T1-features['dIncorporationDate']
features.loc[features['dAccountsOutstandingTime']<0, 'dAccountsOutstandingTime'] = 0 
features.loc[features['dConfStmtOutstandingTime']<0, 'dConfStmtOutstandingTime'] = 0 
features.loc[features['dReturnsOutstandingTime']<0, 'dReturnsOutstandingTime'] = 0
features.drop(['dAccountsNextDueDate','dConfStmtNextDueDate','dReturnsNextDueDate','dIncorporationDate'],axis = 1,inplace = True)
features.head()

,CompanyNumber,oac2,CountryOfOrigin,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,LimitedPartnershipsNumGenPartners,LimitedPartnershipsNumLimPartners,SIC1,eCompanyCategory,eCompanyStatus,eAccountsAccountCategory,namechanged,namechanged2,class_1,dAccountsOutstandingTime,dConfStmtOutstandingTime,dReturnsOutstandingTime,OperatingTime
0,08209948,8A,United Kingdom,0,0,0,0,0,99.0,15,2,3,0,0,False,0.0,0.0,1.467655,6.303720
1,11006939,3B,United Kingdom,0,0,0,0,0,62.0,15,2,10,0,0,False,0.0,0.0,0.000000,1.301370
2,SC421617,2C,United Kingdom,0,0,0,0,0,70.0,15,2,13,0,0,False,0.0,0.0,0.482192,6.317382
3,09152972,1C,United Kingdom,0,0,0,0,0,58.0,15,2,9,0,0,False,0.0,0.0,1.505906,4.583562
4,07687209,5B,United Kingdom,1,1,0,0,0,70.0,15,2,12,1,0,False,0.0,0.0,0.591781,7.668493


In [8]:
features.drop(['dConfStmtOutstandingTime','CountryOfOrigin','LimitedPartnershipsNumGenPartners','LimitedPartnershipsNumLimPartners','eCompanyStatus'],axis = 1,inplace = True)

In [9]:
features = pd.get_dummies(features, columns=['SIC1', 'oac2','eCompanyCategory','eAccountsAccountCategory'])

In [10]:
features.head()

,CompanyNumber,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,namechanged,namechanged2,class_1,dAccountsOutstandingTime,dReturnsOutstandingTime,OperatingTime,SIC1_1.0,SIC1_2.0,SIC1_3.0,SIC1_5.0,SIC1_6.0,SIC1_7.0,SIC1_8.0,SIC1_9.0,SIC1_10.0,SIC1_11.0,SIC1_12.0,SIC1_13.0,SIC1_14.0,SIC1_15.0,SIC1_16.0,SIC1_17.0,SIC1_18.0,SIC1_19.0,SIC1_20.0,SIC1_21.0,SIC1_22.0,SIC1_23.0,SIC1_24.0,SIC1_25.0,SIC1_26.0,SIC1_27.0,SIC1_28.0,SIC1_29.0,SIC1_30.0,SIC1_31.0,SIC1_32.0,SIC1_33.0,SIC1_35.0,SIC1_36.0,SIC1_37.0,SIC1_38.0,SIC1_39.0,SIC1_41.0,SIC1_42.0,SIC1_43.0,SIC1_45.0,SIC1_46.0,SIC1_47.0,SIC1_49.0,SIC1_50.0,SIC1_51.0,SIC1_52.0,SIC1_53.0,SIC1_55.0,SIC1_56.0,SIC1_58.0,SIC1_59.0,SIC1_60.0,SIC1_61.0,SIC1_62.0,SIC1_63.0,SIC1_64.0,SIC1_65.0,SIC1_66.0,SIC1_68.0,SIC1_69.0,SIC1_70.0,SIC1_71.0,SIC1_72.0,SIC1_73.0,SIC1_74.0,SIC1_75.0,SIC1_77.0,SIC1_78.0,SIC1_79.0,SIC1_80.0,SIC1_81.0,SIC1_82.0,SIC1_84.0,SIC1_85.0,SIC1_86.0,SIC1_87.0,SIC1_88.0,SIC1_90.0,SIC1_91.0,SIC1_92.0,SIC1_93.0,SIC1_94.0,SIC1_95.0,SIC1_96.0,SIC1_97.0,SIC1_98.0,SIC1_99.0,oac2_1A,oac2_1B,oac2_1C,oac2_2A,oac2_2B,oac2_2C,oac2_2D,oac2_3A,oac2_3B,oac2_3C,oac2_3D,oac2_4A,oac2_4B,oac2_4C,oac2_5A,oac2_5B,oac2_6A,oac2_6B,oac2_7A,oac2_7B,oac2_7C,oac2_7D,oac2_8A,oac2_8B,oac2_8C,oac2_8D,eCompanyCategory_15,eCompanyCategory_19,eAccountsAccountCategory_0,eAccountsAccountCategory_1,eAccountsAccountCategory_2,eAccountsAccountCategory_3,eAccountsAccountCategory_4,eAccountsAccountCategory_5,eAccountsAccountCategory_6,eAccountsAccountCategory_7,eAccountsAccountCategory_8,eAccountsAccountCategory_9,eAccountsAccountCategory_10,eAccountsAccountCategory_11,eAccountsAccountCategory_12,eAccountsAccountCategory_13,eAccountsAccountCategory_14,eAccountsAccountCategory_15
0,08209948,0,0,0,0,0,False,0.0,1.467655,6.303720,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,11006939,0,0,0,0,0,False,0.0,0.000000,1.301370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,SC421617,0,0,0,0,0,False,0.0,0.482192,6.317382,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,09152972,0,0,0,0,0,False,0.0,1.505906,4.583562,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,07687209,1,1,0,1,0,False,0.0,0.591781,7.668493,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [11]:
pickle.dump(features, open("processed_data.pkl", "wb" ) )